### Question 16

In [19]:
import numpy as np
from scipy.linalg import expm

def build_generator_Q(k: int = 12) -> np.ndarray:

    S = np.array([
        [-1.0,   1.0,  0.0,  0.0],   
        [ 0.0,  -1.0,  0.9,  0.1],   
        [ 0.0,   0.0, -1.0,  0.0],   
        [ 0.0,   0.0,  0.0, -1.0/11.0] 
    ])

    s = np.array([0.0, 0.0, 1.0, 1.0/11.0])  

    n_levels = k            
    dim = 4 * n_levels + 1    
    absorbing = dim - 1

    Q = np.zeros((dim, dim))

    def idx(phase: int, n: int) -> int:
        return 4 * n + phase

    for n in range(n_levels):
        Q[4*n:4*n+4, 4*n:4*n+4] = S

        if n < n_levels - 1:
            for phase in range(4):
                if s[phase] != 0.0:
                    Q[idx(phase, n), idx(0, n+1)] += s[phase]
        else:
            for phase in range(4):
                if s[phase] != 0.0:
                    Q[idx(phase, n), absorbing] += s[phase]

    return Q

def prob_N_less_than_k(t_months: float = 60.0, k: int = 12) -> float:

    Q = build_generator_Q(k=k)
    dim = Q.shape[0]
    absorbing = dim - 1

    p0 = np.zeros(dim)
    p0[0] = 1.0

    pt = p0 @ expm(Q * t_months)
    return 1.0 - pt[absorbing]

if __name__ == "__main__":
    t_months = 60.0  # 5 years
    k = 12   
    p = prob_N_less_than_k(t_months=t_months, k=k)
    print(f"P(N(5 years) < 12) = {p:.6f}")


P(N(5 years) < 12) = 0.189481


### Question 17

In [24]:
import scipy
S = np.array([
    [-1.0,  1.0,  0.0,  0.0],
    [ 0.0, -1.0,  0.9,  0.1],
    [ 0.0,  0.0, -1.0,  0.0],
    [ 0.0,  0.0,  0.0, -1.0/11.0]
], dtype=float)

alpha = np.array([[1.0, 0.0, 0.0, 0.0]])  # initial phase distribution for a *fresh* cycle
one = np.ones((4, 1))


inv_minus_S = np.linalg.inv(-S)
mu = float(alpha @ inv_minus_S @ one)

pi_unnormalized = alpha @ inv_minus_S
pi = pi_unnormalized / float(pi_unnormalized @ one)


t = 6.0  # months
surv_R = float(pi @ scipy.linalg.expm(S * t) @ one)          # P(R > t)
prob_within_t = 1.0 - surv_R                    # P(R <= t)

print("mu (months) =", mu)
print("pi =", pi)
print(f"P(next termination within {t} months) =", prob_within_t)


mu (months) = 4.0
pi = [[0.25  0.25  0.225 0.275]]
P(next termination within 6.0 months) = 0.7887969844346261


/var/folders/g5/ds02r9td3kd859fn3kytf2nr0000gn/T/ipykernel_71816/1262766756.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mu = float(alpha @ inv_minus_S @ one)
/var/folders/g5/ds02r9td3kd859fn3kytf2nr0000gn/T/ipykernel_71816/1262766756.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pi = pi_unnormalized / float(pi_unnormalized @ one)
/var/folders/g5/ds02r9td3kd859fn3kytf2nr0000gn/T/ipykernel_71816/1262766756.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1

### Question 18

In [ ]:
from sympy import *
l = latex(Matrix([[-1,1,0,0,0,0,0,0],[0,-1,0.9,0.1,0,0,0,0],[0.65,0,-1,0,0.35,0,0,0],[S(0.65)/11,0,0,-S(1)/11,S(0.35)/11,0,0,0],[0,0,0,0,-1,1,0,0],[0,0,0,0,0,-1,0.9,0.1],[1,0,0,0,0,0,-1,0],[S(1)/11,0,0,0,0,0,0,-S(1)/11]]))
print(l)

\left[\begin{matrix}-1 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & -1 & 0.9 & 0.1 & 0 & 0 & 0 & 0\\0.65 & 0 & -1 & 0 & 0.35 & 0 & 0 & 0\\0.0590909090909091 & 0 & 0 & - \frac{1}{11} & 0.0318181818181818 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -1 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & -1 & 0.9 & 0.1\\1 & 0 & 0 & 0 & 0 & 0 & -1 & 0\\\frac{1}{11} & 0 & 0 & 0 & 0 & 0 & 0 & - \frac{1}{11}\end{matrix}\right]


#Question 20
## Here we remove the idle to busy jumps to model time busy -> idle -> busy

In [28]:
T = (Matrix([[-1,1,0,0,0,0,0,0],[0,-1,0.9,0.1,0,0,0,0],[0.65,0,-1,0,0,0,0,0],[S(0.65)/11,0,0,-S(1)/11,0,0,0,0],[0,0,0,0,-1,1,0,0],[0,0,0,0,0,-1,0.9,0.1],[1,0,0,0,0,0,-1,0],[S(1)/11,0,0,0,0,0,0,-S(1)/11]]))
alpha_busy = Matrix([0,0,0,0,1,0,0,0]).T
ones = Matrix([[1],[1],[1],[1],[1],[1],[1],[1]])
U = (-T).inv()
m1 = alpha_busy @ U @ ones #PH-type paper Section 1.4.3 corollary 1 #first order moment
m2 = 2*alpha_busy @ U @ U @ ones #second order moment
var = m2 - m1**2
print("mean",m1), print("var",var)

mean Matrix([[15.4285714285714]])
var Matrix([[177.469387755102]])


(None, None)

In [24]:
ones.shape,alpha_busy.shape,T.shape


((8, 1), (8, 1), (8, 8))